In [310]:
import numpy as np
import numpy.linalg as la

import logging
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import itertools
import argh
import cvxopt

import scipy.linalg as la

In [311]:
class CqkProblem:
    def __init__(self, r, n, d, a, b, low, up):
        self.n = n
        self.r = r
        self.d = list(d)
        self.a = list(a)
        self.b = list(b)
        self.low = list(low)
        self.up = list(up)

In [312]:

def initial_lambda(p, lamb):
    s0=0.0
    q0=0.0
    slopes = []
    for i in range(p.n):
        slopes.append((p.b[i]/p.d[i])*p.b[i])
        s0 = s0 + (p.a[i] * p.b[i]) / p.d[i]
        q0 = q0 + (p.b[i] * p.b[i]) / p.d[i]
    lamb = (p.r-s0)/q0
    return lamb, slopes

In [516]:
def phi_lambda(p,lamb,phi,deriv,slopes,r):
    deriv = 0.0
    phi = r * -1
    x = []
    
    for i in range(p.n):
        
        x.append( (p.b[i] * lamb + p.a[i])/p.d[i])

        if x[i] < p.low[i]:
            x[i] = p.low[i]
        elif x[i] > p.up[i]:
            x[i] = p.up[i]
        else:
            deriv = deriv + slopes[i];
        phi = phi + p.b[i] * x[i];
    return deriv, phi, x

In [561]:
MAX_IT = 20
INFINITO_NEGATIVO = -999999999;
INFINITO_POSITIVO = 999999999;
def newton(p):
    phi = 0
    lamb = 0
    lambs = []
    alfa = INFINITO_NEGATIVO;
    beta = INFINITO_POSITIVO;
    phi_alfa = 0.0;
    phi_beta = 0.0;
    deriv = 0
    x = []
    r = p.r
    
    lamb, slopes = initial_lambda(p,lamb)
    lambs.append(lamb)
    deriv, phi, x = phi_lambda(p,lamb,phi,deriv,slopes,r)
    it = 1
    while phi != 0.0 and it <= MAX_IT:
        if phi > 0:
#             print("positivo")
            beta = lamb
            lambda_n = 0.0
            if deriv > 0.0:
                
                lambda_n = lamb - (phi/deriv)
                if abs(lambda_n - lamb) <= 0.00000000001:
                    phi = 0.0
                    break
                if lambda_n > alfa:
                    lamb = lambda_n
                else:
                    print("aqui")
                    phi_beta = phi;
#                     lamb = secant(p,x,alfa,beta,phi_alfa,phi_beta,r);
#             if deriv == 0.0:
#                 lamb = breakpoint_to_the_left(p,lamb);
#                 if lamb <= INFINITO_NEGATIVO or lamb >= INFINITO_POSITIVO:
#                     break
                
        else:
#             print("negativo")
            alfa = lamb;
            lambda_n = 0.0;

            if deriv > 0.0:
                lambda_n = lamb - (phi/deriv)
                if abs(lambda_n - lamb) <= 0.00000000001:
                    phi = 0.0
                    break
                
                if lambda_n < beta:
                    lamb = lambda_n
                else:
                    print("aqui")
                    phi_alfa = phi;
#                     lamb = secant(p,x,alfa,beta,phi_alfa,phi_beta,r);
            if deriv == 0.0:
                print("aqui 2")
#                 lamb = breakpoint_to_the_right(p,lamb)
#                 if lamb <= INFINITO_NEGATIVO or lamb >= INFINITO_POSITIVO:
#                     break
        
        
        deriv, phi, x = phi_lambda(p,lamb,phi,deriv,slopes,r)
        lambs.append(lamb)
        it = it + 1
        
    if phi == 0.0:
        return it, x, slopes,lambs
    elif alfa == beta:
        return -1, x, slopes,lambs
    else:
        return -2, x, slopes,lambs
    

In [562]:

def linear():
    return lambda x, y: np.inner(x, y)

In [563]:
d = []
a = []
up = []
low = []
bb = []
r = 0
class SVMTrainer(object):
    def __init__(self, kernel, c):
        self._kernel = kernel
        self._c = c

    def train(self, X, y):
        """Given the training features X with labels y, returns a SVM
        predictor representing the trained SVM.
        """
        lagrange_multipliers = self._compute_multipliers(X, y)
#         return self._construct_predictor(X, y, lagrange_multipliers)

    def _gram_matrix(self, X):
        n_samples, n_features = X.shape
        K = np.zeros((n_samples, n_samples))
        # TODO(tulloch) - vectorize
        for i, x_i in enumerate(X):
            for j, x_j in enumerate(X):
                K[i, j] = self._kernel(x_i, x_j)
        return K

    def _construct_predictor(self, X, y, lagrange_multipliers):
        support_vector_indices = \
            lagrange_multipliers > MIN_SUPPORT_VECTOR_MULTIPLIER

        support_multipliers = lagrange_multipliers[support_vector_indices]
        support_vectors = X[support_vector_indices]
        support_vector_labels = y[support_vector_indices]

        # http://www.cs.cmu.edu/~guestrin/Class/10701-S07/Slides/kernels.pdf
        # bias = y_k - \sum z_i y_i  K(x_k, x_i)
        # Thus we can just predict an example with bias of zero, and
        # compute error.
        bias = np.mean(
            [y_k - SVMPredictor(
                kernel=self._kernel,
                bias=0.0,
                weights=support_multipliers,
                support_vectors=support_vectors,
                support_vector_labels=support_vector_labels).predict(x_k)
             for (y_k, x_k) in zip(support_vector_labels, support_vectors)])

        return SVMPredictor(
            kernel=self._kernel,
            bias=bias,
            weights=support_multipliers,
            support_vectors=support_vectors,
            support_vector_labels=support_vector_labels)

    def _compute_multipliers(self, X, y):
        n_samples, n_features = X.shape

        K = self._gram_matrix(X)

        # Solves
        # min 1/2 x^T P x + q^T x
        # s.t.
        #  Gx \coneleq h
        #  Ax = b

        P = cvxopt.matrix(np.outer(y, y) * K)
        q = cvxopt.matrix(-1 * np.ones(n_samples))

        # -a_i \leq 0
        # TODO(tulloch) - modify G, h so that we have a soft-margin classifier
        G_std = cvxopt.matrix(np.diag(np.ones(n_samples) * -1))
        h_std = cvxopt.matrix(np.zeros(n_samples))

        # a_i \leq c
        G_slack = cvxopt.matrix(np.diag(np.ones(n_samples)))
        h_slack = cvxopt.matrix(np.ones(n_samples) * self._c)

        G = cvxopt.matrix(np.vstack((G_std, G_slack)))
        h = cvxopt.matrix(np.vstack((h_std, h_slack)))

        A = cvxopt.matrix(y, (1, n_samples))
        b = cvxopt.matrix(0.0)

        solution = cvxopt.solvers.qp(P, q, G, h, A, b)
        
        evals, evecs = la.eig(K)
        print(evals)
        
        global d
        d = [i.real for i in evals]
        global a
        a= [i*-1 for i in np.ones(len(d)) ]
        global up
        up = [i/10 for i in np.ones(len(d)) ]
        global low
        low = [i for i in  np.zeros(len(d))]
        global bb
        bb= [i for i in A ]
        global r
        r = 0
        
        print("P",d)
        
        
        
        print("q", q)
        print("h", up)
        print("A", A)
        print("b", r)
        print("low",low)
        print("solution['x']",solution['x'])

        # Lagrange multipliers
        return np.ravel(solution['x'])


In [564]:
def example(num_samples=10, num_features=2, grid_size=20, filename="svm.pdf"):
    samples = np.matrix(np.random.normal(size=num_samples * num_features)
                        .reshape(num_samples, num_features))
    labels = 2 * (samples.sum(axis=1) > 0) - 1.0
    trainer = SVMTrainer(linear(), 0.1)
    predictor = trainer.train(samples, labels)


In [565]:
example()

     pcost       dcost       gap    pres   dres
 0: -1.1004e+00 -1.9093e+00  4e+01  6e+00  2e-16
 1: -6.6218e-01 -1.6897e+00  3e+00  3e-01  4e-16
 2: -4.8365e-01 -8.8168e-01  4e-01  7e-17  8e-16
 3: -4.9913e-01 -5.5284e-01  5e-02  4e-17  3e-16
 4: -5.1633e-01 -5.2630e-01  1e-02  6e-17  1e-16
 5: -5.2091e-01 -5.2221e-01  1e-03  6e-17  2e-16
 6: -5.2146e-01 -5.2153e-01  6e-05  9e-17  2e-16
 7: -5.2150e-01 -5.2151e-01  3e-06  6e-17  3e-16
 8: -5.2151e-01 -5.2151e-01  3e-08  6e-17  2e-16
Optimal solution found.
[ 1.45690541e+01+0.00000000e+00j  3.33066907e-15+0.00000000e+00j
  5.40977436e+00+0.00000000e+00j -6.97689462e-16+0.00000000e+00j
  2.81779480e-16+2.37389848e-17j  2.81779480e-16-2.37389848e-17j
 -1.61072413e-16+0.00000000e+00j  5.11156444e-17+1.43188026e-17j
  5.11156444e-17-1.43188026e-17j -3.84200046e-17+0.00000000e+00j]
P [14.569054134073562, 3.3306690738754696e-15, 5.409774359444797, -6.976894615962706e-16, 2.817794797333256e-16, 2.817794797333256e-16, -1.610724130402916e-16, 5

In [566]:
print(r,len(d),d,a,bb,low,up)
def generate_cqk_problem(n):
    return CqkProblem( r, n, d, a, bb, low, up)

0 10 [14.569054134073562, 3.3306690738754696e-15, 5.409774359444797, -6.976894615962706e-16, 2.817794797333256e-16, 2.817794797333256e-16, -1.610724130402916e-16, 5.1115644369403624e-17, 5.1115644369403624e-17, -3.842000457301277e-17] [-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0] [-1.0, 1.0, -1.0, -1.0, -1.0, 1.0, -1.0, -1.0, -1.0, 1.0] [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]


In [567]:
pp = generate_cqk_problem(len(d))

In [568]:
it,x, slopes,lambs = newton(pp)

aqui 2
aqui 2
aqui 2
aqui 2
aqui 2
aqui 2
aqui 2
aqui 2
aqui 2
aqui 2
aqui 2
aqui 2
aqui 2
aqui 2
aqui 2
aqui 2
aqui 2
aqui 2
aqui 2
aqui 2


In [569]:
it, lambs

(-2,
 [-4.450598813345339,
  -4.450598813345339,
  -4.450598813345339,
  -4.450598813345339,
  -4.450598813345339,
  -4.450598813345339,
  -4.450598813345339,
  -4.450598813345339,
  -4.450598813345339,
  -4.450598813345339,
  -4.450598813345339,
  -4.450598813345339,
  -4.450598813345339,
  -4.450598813345339,
  -4.450598813345339,
  -4.450598813345339,
  -4.450598813345339,
  -4.450598813345339,
  -4.450598813345339,
  -4.450598813345339,
  -4.450598813345339])

In [533]:
x

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [437]:
lambs

[-1.693873964497774]

In [ ]:
evals, evecs = la.eig(M)
print(evals)